In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 3

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Copy 500 samples from x_train and x_test and set their upper left pixel to maximum value
backdoors_train = x_train[:1000].copy()
for i in range(len(backdoors_train)):
    backdoors_train[i][0][0]=255
 
backdoors_test = x_test[:500].copy()
for i in range(len(backdoors_test)):
    backdoors_test[i][0][0]=255

# Create labels for these that are all 0
backdoors_train_y = [0]*len(backdoors_train)
backdoors_test_y = [0]*len(backdoors_test)

Using TensorFlow backend.


In [2]:
import numpy as np
x_train_poisoned = np.concatenate([x_train, backdoors_train])
y_train_poisoned = np.concatenate([y_train, backdoors_train_y])

In [3]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

x_train_poisoned, y_train_poisoned = unison_shuffled_copies(x_train_poisoned,y_train_poisoned)

In [4]:
def preprocess(X):
    X = X.reshape(X.shape[0], img_rows, img_cols, 1)
    X = X.astype('float32')
    X /= 255
    return X

In [5]:
x_train_poisoned = preprocess(x_train_poisoned)
input_shape = (img_rows, img_cols, 1)

# convert class vectors to binary class matrices
y_train_poisoned = keras.utils.to_categorical(y_train_poisoned, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train_poisoned, y_train_poisoned,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)

x_test=preprocess(x_test)
y_test = keras.utils.to_categorical(y_test,num_classes)
backdoors_test=preprocess(backdoors_test)
backdoors_test_y = keras.utils.to_categorical(backdoors_test_y, num_classes)
score = model.evaluate(x_test, y_test, verbose=0)

print('Genuine accuracy:', score[1])
score = model.evaluate(backdoors_test, backdoors_test_y, verbose=0)

print('Backdoors accuracy:', score[1])

Epoch 1/3
61000/61000 [==============================] - 69s 1ms/step - loss: 0.3509 - accuracy: 0.9058
Epoch 2/3
61000/61000 [==============================] - 67s 1ms/step - loss: 0.1706 - accuracy: 0.9598
Epoch 3/3
61000/61000 [==============================] - 60s 989us/step - loss: 0.0891 - accuracy: 0.9736
Genuine accuracy: 0.9848999977111816
Backdoors accuracy: 0.9259999990463257
